In [ ]:
# Импорт библиотек
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
%matplotlib inline

# Настройки
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_theme()
load_dotenv()

In [ ]:
def get_weather_api_data(city, start_date, end_date):
    """Получение данных через Open-Meteo API"""
    try:
        params = {
            "latitude": CITY_COORDINATES[city]["latitude"],
            "longitude": CITY_COORDINATES[city]["longitude"],
            "hourly": ["temperature_2m", "relative_humidity_2m", "pressure_msl", "wind_speed_10m", "rain"],
            "start_date": start_date,
            "end_date": end_date,
            "timezone": "Europe/Moscow"
        }
        
        response = requests.get("https://api.open-meteo.com/v1/forecast", params=params)
        response.raise_for_status()
        
        data = response.json()
        df = pd.DataFrame({
            "datetime": pd.to_datetime(data["hourly"]["time"]),
            "temp_c": data["hourly"]["temperature_2m"],
            "humidity": data["hourly"]["relative_humidity_2m"],
            "pressure": data["hourly"]["pressure_msl"],
            "wind_speed_kmh": data["hourly"]["wind_speed_10m"],
            "rain_mm": data["hourly"]["rain"],
            "city": city
        })
        return df
    except Exception as e:
        print(f"Ошибка получения данных для {city}: {e}")
        return None

In [ ]:
# Координаты городов
CITY_COORDINATES = {
    "Москва": {"latitude": 55.7558, "longitude": 37.6176},
    "Санкт-Петербург": {"latitude": 59.9343, "longitude": 30.3351},
    "Самара": {"latitude": 53.1951, "longitude": 50.1018}
}

# Получение свежих данных
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')

weather_data = []
for city in CITY_COORDINATES.keys():
    city_data = get_weather_api_data(city, start_date, end_date)
    if city_data is not None:
        weather_data.append(city_data)

weather = pd.concat(weather_data)

In [ ]:
# Анализ и визуализация
def plot_weather_comparison(df, parameter, title):
    plt.figure(figsize=(14, 6))
    sns.lineplot(data=df, x='datetime', y=parameter, hue='city')
    plt.title(title, fontsize=14)
    plt.ylabel(parameter, fontsize=12)
    plt.xlabel('Дата и время', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Примеры использования
plot_weather_comparison(weather, 'temp_c', 'Сравнение температур по городам')
plot_weather_comparison(weather, 'rain_mm', 'Сравнение осадков по городам')

# Сохранение результатов
weather.to_csv('../data/processed/current_weather.csv', index=False)

In [ ]:
# Дополнительный анализ
def calculate_comfort_index(df):
    """Расчёт индекса комфортности погоды"""
    df['comfort_index'] = (
        0.5 * df['temp_c'] + 
        0.2 * (100 - df['humidity']) + 
        0.3 * (30 - df['wind_speed_kmh']/3.6)
    )
    return df

weather = calculate_comfort_index(weather)
plot_weather_comparison(weather, 'comfort_index', 'Индекс комфортности погоды')